In [1]:
import pandas
import csv 
import numpy as np
from datasketch import MinHash, MinHashLSH

In [2]:
# dump the CSV file into a numpy array. 
csv_reader = pandas.read_csv('data/string_study.csv', delimiter=',', header=None)
final_arr = csv_reader.values
final_arr = np.array(final_arr)

In [3]:
final_arr[1:4]

array([['1524387149611', 'attach-volume', 'wig_project_003',
        "2018-04-22 10:52:29.611 17979 ERROR rallytester.rallytester [-] [nova attach-volume wig_project_003] Task failed:  Rally tired waiting 1440.00 seconds for Server rally-9db3-aPln:176b6fd3-8a2d-4b60-a5bd-73575c4ad161 to become ('ACTIVE') current status BUILD",
        '2018-04-22 08:00:00', "waiting for Server to become ('ACTIVE')",
        "Rally tired waiting 1440.00 seconds for Server rally-9db3-aPln:176b6fd3-8a2d-4b60-a5bd-73575c4ad161 to become ('ACTIVE') current status BUILD"],
       ['1524387185889', 'boot-from-volume-linux', 'gva_shared_016',
        '2018-04-22 10:53:05.889 25667 ERROR rallytester.rallytester [-] [nova boot-from-volume-linux gva_shared_016] Task failed:  Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, instances (HTTP 403) (Request-ID: req-23b5fe81-0bab-4f42-84e9-97e3e6847c04)',
        '2018-04-22 08:00:00',
        'Quota exceeded for cores, inst

In [6]:
# extract all the raw_messsages from the error logs
raw_msgs = final_arr[0:, -1]
raw_msgs.shape
raw_msgs[1:2]

array(['Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, instances (HTTP 403) (Request-ID: req-23b5fe81-0bab-4f42-84e9-97e3e6847c04)'],
      dtype=object)

In [8]:
# Generate the set of words of all raw messages
message_set = []
for item in raw_msgs:
    set_of_words = set(item.split(' '))
    message_set.append(set_of_words)
message_set[1:2]

[{'(HTTP',
  '(Request-ID:',
  '1,',
  '10',
  '10,',
  '403)',
  'Quota',
  'Requested',
  'already',
  'but',
  'cores,',
  'exceeded',
  'for',
  'instances',
  'instances:',
  'of',
  'req-23b5fe81-0bab-4f42-84e9-97e3e6847c04)',
  'used'}]

In [9]:
# Generate MinHash instances for all messages
m = []
for i in range (0, len(message_set)):
    m_temp = MinHash(num_perm=128)
    m.append(m_temp)

In [10]:
i = 0
for message in message_set:
    for word in message:
        m[i].update(word.encode('utf-8'))
    i = i + 1

In [11]:
# Create an LSH index 
lsh = MinHashLSH(threshold=.4, num_perm=128)
for i in range (1, len(m)):
    string = "m" + str(i)
    lsh.insert(string, m[i])

In [14]:
# Find all the messages similar to the fist message with a specified threshold
result = lsh.query(m[0])
len(result)

6395